In [1]:
import pickle

result_casa = pickle.load(open("./result_casa.pickle", 'rb'))
result_hoasa = pickle.load(open("./result_hoasa.pickle", 'rb'))

In [20]:
with open("casa_absa-prosa_data.txt", 'r') as fp:
    text_casa = fp.read().replace("####[([0], [1], 'POS')]", '\n').strip().split("\n\n")

with open("hoasa_absa-airy_data.txt", 'r') as fp:
    text_hoasa = fp.read().replace("####[([0], [1], 'POS')]", '\n').strip().split("\n\n")

In [22]:
len(result_casa[0]) == len(text_casa)*5

True

In [25]:
res = result_casa[0][0]
res

'[A] mobil nya [O] bagus [S] great [SSEP] [A] mobil nya [O] enak [S] great [SSEP] [A] mobil nya [O] menyenangkan [S] great'

In [47]:
result_casa[0][:5]

['[A] mobil nya [O] bagus [S] great [SSEP] [A] mobil nya [O] enak [S] great [SSEP] [A] mobil nya [O] menyenangkan [S] great',
 '[O] bagus [S] great [A] mobil nya [SSEP] [O] enak [S] great [A] mobil nya [SSEP] [O] menyenangkan [S] great [A] dikendelai',
 '[A] mobil nya [S] great [O] bagus [SSEP] [A] mobil nya [S] great [O] enak [SSEP] [A] mobil nya [S] great [O] sesuai [SSEP] [A] dikendelai [S] great [O] menyenangkan',
 '[O] bagus [A] mobil nya [S] great [SSEP] [O] enak [A] mobil nya [S] great [SSEP] [O] menyenangkan [A] dikendelai [S] great',
 '[S] great [A] mobil nya [O] bagus [SSEP] [S] great [A] mobil nya [O] enak [SSEP] [S] great [A] mobil nya [O] menyenangkan']

In [28]:
res = [el.strip() for el in res.split("[SSEP]")]
res

['[A] mobil nya [O] bagus [S] great',
 '[A] mobil nya [O] enak [S] great',
 '[A] mobil nya [O] menyenangkan [S] great']

In [38]:
text = text_casa[0].lower()
text

'saya memakai honda jazz gk5 tahun 2014 ( pertama meluncur ) . mobil nya bagus dan enak sesuai moto nya menyenangkan untuk dikendarai'

In [33]:
import editdistance

In [35]:
editdistance.distance("mobil nya", "Mobil nya")

1

In [37]:
sentiment_map = {'great' : 'POS', 'bad' : 'NEG', 'ok' : 'NEU'}

In [78]:
def norm_editdistance(text1, text2):
    ed = editdistance.distance(text1, text2)
    return ed/max(len(text1), len(text2))

def find_index(phrase, sentence):
    tokenized_phrase = phrase.split()
    tokenized_sentence = sentence.split()

    if len(tokenized_phrase) == 0 or len(tokenized_sentence) == 0:
        return None

    for i in range(0, len(tokenized_sentence) - len(tokenized_phrase) + 1):
        window = tokenized_sentence[i:i+len(tokenized_phrase)]

        joined_window = ' '.join(window)

        if norm_editdistance(joined_window, phrase) < 0.5:
            return [i+j for j in range(len(tokenized_phrase))]
    return None

In [79]:
find_index("mobil nya", text)

In [80]:
res[0].index("[S]")

24

In [81]:
res[0][14:24]

'[O] bagus '

In [82]:
labels_casa = []

for i in range(len(text_casa)):
    text = text_casa[i].lower().strip()

    if len(text) == 0:
        labels_casa.append("[]")
    predicted = result_casa[0][(i*5):(i+1)*5]
    predicted = " [SSEP] ".join(predicted)
    predicted = [el.strip() for el in predicted.split("[SSEP]")]

    tuples = []
    for el in predicted:
        try:
            a_id = el.index("[A]")
            o_id = el.index("[O]")
            s_id = el.index("[S]")
        except:
            continue

        order = sorted([a_id, o_id, s_id])

        # ASPECT

        end_a_id = order.index(a_id)+1
        end_a_id = len(el) if end_a_id > 2 else order[end_a_id]

        aspect = el[
            a_id+3:end_a_id
        ].strip()

        # OPINION

        end_o_id = order.index(o_id)+1
        end_o_id = len(el) if end_o_id > 2 else order[end_o_id]

        opinion = el[
            o_id+3:end_o_id
        ].strip()

        # SENTIMENT

        end_s_id = order.index(s_id)+1
        end_s_id = len(el) if end_s_id > 2 else order[end_s_id]

        sentiment = el[
            s_id+3:end_s_id
        ].strip()

        aspect_token_id = find_index(aspect, text)
        opinion_token_id = find_index(opinion, text)
        sentiment_result = None if sentiment not in sentiment_map else sentiment_map[sentiment]

        if aspect_token_id is None or opinion_token_id is None or sentiment_result is None:
            continue
        t = (aspect_token_id, opinion_token_id, sentiment_result)
        if t not in tuples:
            tuples.append(t)
    labels_casa.append(tuples)

In [83]:
labels_hoasa = []

for i in range(len(text_hoasa)):
    text = text_hoasa[i].lower().strip()
    predicted = result_hoasa[0][(i*5):(i+1)*5]
    predicted = " [SSEP] ".join(predicted)
    predicted = [el.strip() for el in predicted.split("[SSEP]")]

    tuples = []
    for el in predicted:
        try:
            a_id = el.index("[A]")
            o_id = el.index("[O]")
            s_id = el.index("[S]")
        except:
            continue

        order = sorted([a_id, o_id, s_id])

        # ASPECT

        end_a_id = order.index(a_id)+1
        end_a_id = len(el) if end_a_id > 2 else order[end_a_id]

        aspect = el[
            a_id+3:end_a_id
        ].strip()

        # OPINION

        end_o_id = order.index(o_id)+1
        end_o_id = len(el) if end_o_id > 2 else order[end_o_id]

        opinion = el[
            o_id+3:end_o_id
        ].strip()

        # SENTIMENT

        end_s_id = order.index(s_id)+1
        end_s_id = len(el) if end_s_id > 2 else order[end_s_id]

        sentiment = el[
            s_id+3:end_s_id
        ].strip()

        aspect_token_id = find_index(aspect, text)
        opinion_token_id = find_index(opinion, text)
        sentiment_result = None if sentiment not in sentiment_map else sentiment_map[sentiment]

        if aspect_token_id is None or opinion_token_id is None or sentiment_result is None:
            continue

        t = (aspect_token_id, opinion_token_id, sentiment_result)
        if t not in tuples:
            tuples.append(t)
    labels_hoasa.append(tuples)

In [84]:
labels_hoasa[:5]

[[([0], [1], 'NEG')],
 [([5], [6], 'NEG'), ([5], [7], 'NEG')],
 [([0], [1], 'POS'), ([0], [2], 'POS'), ([4], [5, 6], 'NEG')],
 [([0], [1], 'POS'),
  ([4, 5], [10, 11], 'POS'),
  ([13, 14], [18, 19], 'NEG'),
  ([21], [23], 'NEG')],
 [([0, 1], [2, 3], 'NEG'),
  ([4], [5], 'NEG'),
  ([4], [3], 'NEG'),
  ([8, 9], [10, 11], 'NEG'),
  ([12], [13, 14], 'NEG')]]

In [85]:
text_hoasa[0]

'kebersihan kurang...'

In [86]:
result_hoasa[0][:5]

['[A] kebersihan [O] kurang [S] bad',
 '[O] kurang [S] bad [A] kebersihan',
 '[A] kebersihan [S] bad [O] kurang',
 '[O] kurang [A] kebersihan [S] bad',
 '[S] bad [A] kebersihan [O] kurang']

In [88]:
len(labels_hoasa) == len(text_hoasa)

True

In [89]:
casa = '\n'.join([
    f"{t}####{l}" for t, l in zip(text_casa, labels_casa)
])

hoasa = '\n'.join([
    f"{t}####{l}" for t, l in zip(text_hoasa, labels_hoasa)
])

In [91]:
print(casa)

Saya memakai Honda Jazz GK5 tahun 2014 ( pertama meluncur ) . Mobil nya bagus dan enak sesuai moto nya menyenangkan untuk dikendarai####[([12, 13], [14], 'POS'), ([12, 13], [16], 'POS'), ([12, 13], [20], 'POS'), ([22], [20], 'POS'), ([12, 13], [17], 'POS')]
Avanza kenapa jadi boros bensin begini dah ah. Baru diisi sudah mau setengah saja .####[]
saran ku dan pengalaman ku , mending beli mobil niaga L300 atau canter . irit dan bandel .####[([7, 8], [6], 'POS'), ([7, 8], [16], 'POS'), ([7, 8, 9, 10, 11, 12], [6], 'POS'), ([16], [16], 'POS')]
Dari segi harga juga pajero lebih mahal 30 jutaan dibanding fortuner####[([2], [5, 6, 7, 8], 'NEG')]
Kalo menurut gw enak pajero si####[([4], [3], 'POS')]
CH - R versi negara ini kurang worth it soal fitur dan kelengkapan####[([0, 1, 2], [6, 7], 'NEG'), ([10], [6, 7], 'NEG'), ([12], [6, 7], 'NEG'), ([12], [6], 'NEG')]
@LinaRadj kapan-kapan coba bengkel daihatsu , mereka mau kok . Servis lebih bagus .####[([10], [11, 12], 'POS')]
Mending confero harga

In [92]:
with open("casa.txt", 'w') as fp:
    fp.write(casa)

with open("hoasa.txt", 'w') as fp:
    fp.write(hoasa)